# Appunti per comunicazione

* `$V` versione
* `$R0` set-point
* `$R1` parametro P
* `$R2` parametro I
* `$R3` parametro D
* `$W` RUN
* `$Q` STOP

In [ ]:
import serial
import time

ser = serial.Serial()
ser.port = "COM7"
ser.baudrate = 115200
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n\r'
        ser.write(message.encode('utf-8'))
    if flagR:
        res = ser.readline().decode()
        return res
    
print(query("$R100?"))

time.sleep(1)
ser.close()



+2.011464e+01

